In [33]:
import os
import datetime
import xlrd
import dataset

In [2]:
test_wbk_file = "/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-8-SN0657.xls"

In [3]:
wbk = xlrd.open_workbook(test_wbk_file)

In [6]:
wbk.sheet_names()

['Impedance Values from Automated',
 'Array Map with Automated Tester',
 'Cerebus mapping']

In [7]:
sht = wbk.sheet_by_name("Cerebus mapping")

In [8]:
sht

In [9]:
[x for x in sht.get_rows()]

[[text:'// Auto generated mapfile',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  text:'R14',
  number:18.0,
  number:14.0],
 [text:'//',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  text:'Array Map with Automated Tester',
  empty:'',
  empty:''],
 [text:"// Data is as follows 'c r b e l'",
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:''],
 [text:'//   c - 0 based column from left to right',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:''],
 [text:'//   r - 0 based row from bottom to top',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:''],
 [text:'//   b - bank name - values can be A, B, C, or D',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:''],
 [text:'//   e - 1 based electrode number within the bank - values can be 1-32',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:'',
  empty:''],
 [text:'//   l - label used to 

In [12]:
id_row = sht.row(12)
id_row

[text:'Cerebus mapping for array 2132-8/SN0657',
 empty:'',
 empty:'',
 empty:'',
 empty:'',
 empty:'',
 text:'elec',
 empty:'']

In [13]:
type(id_row)

list

In [15]:
type(id_row[0])

xlrd.sheet.Cell

In [16]:
id_cell = id_row[0]
id_cell

text:'Cerebus mapping for array 2132-8/SN0657'

In [20]:
id_cell.dump()

ctype: 1
value: 'Cerebus mapping for array 2132-8/SN0657'
xf_index: None


In [11]:
[x for x in sht.row(12)]

[text:'Cerebus mapping for array 2132-8/SN0657',
 empty:'',
 empty:'',
 empty:'',
 empty:'',
 empty:'',
 text:'elec',
 empty:'']

In [22]:
[x.dump() for x in sht.row(13)]

ctype: 2
value: 8.0
xf_index: None
ctype: 2
value: 9.0
xf_index: None
ctype: 1
value: 'A'
xf_index: None
ctype: 1
value: '14'
xf_index: None
ctype: 1
value: 'elec99'
xf_index: None
ctype: 1
value: 'A 1'
xf_index: None
ctype: 2
value: 93.0
xf_index: None
ctype: 2
value: 99.0
xf_index: None


[None, None, None, None, None, None, None, None]

In [31]:
def dated_file_name(extension, prefix):
    filename = prefix + "_" + datetime.datetime.now().isoformat() + "." + extension
    return filename

In [25]:
data_dir = "/Users/jjpr/dev/scratch/mkgu_scratch/data"

In [32]:
os.path.join(data_dir, dated_file_name("db", "array_meta"))

'/Users/jjpr/dev/scratch/mkgu_scratch/data/array_meta_2018-01-29T15:32:19.911275.db'

In [55]:
class MappingFile(object):
    def __repr__(self):
        return self.__class__.__name__ + str(vars(self))

In [56]:
mf = MappingFile()
mf.xls_path = test_wbk_file
mf

MappingFile{'xls_path': '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-8-SN0657.xls'}

In [38]:
mf_paths = [os.path.join(data_dir, x) for x in os.listdir(data_dir) if x.endswith(".xls")]
mf_paths

['/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-4F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5F.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-5H.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-8E.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-5-SN0656.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2132-8-SN0657.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2466-1-SN0658.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/2467-6_SN0649.xls',
 '/Users/jjpr/dev/scratch/mkgu_scratch/data/5602-2 SN1025-000936.xls']

In [70]:
def is_comment_row(row):
    return str(row[0].value).lstrip().startswith("//")

In [71]:
for mf in mfs:
    sheet = mf.mapping_worksheet
    mf.comment_lines = {i for i in range(sheet.nrows) if is_comment_row(sheet.row(i))}

In [81]:
def get_mapping(i, row):
    return {
        "column": row[0].value, 
        "row": row[1].value, 
        "bank_name": row[2].value,
        "electrode_number_in_bank": row[3].value,
        "electrode_label": row[4].value,
        "index_in_mapping_sheet": i
    }

In [84]:
def get_mapping_file(path):
    mf = MappingFile()
    mf.xls_path = path
    mf.workbook = xlrd.open_workbook(path)
    mf.mapping_worksheet = mf.workbook.sheet_by_name("Cerebus mapping")
    mf.comment_lines = set()
    mf.description_row = None
    mf.mappings = []
    for i, row in enumerate(mf.mapping_worksheet.get_rows()):
        if is_comment_row(row):
            mf.comment_lines.add(i)
        elif mf.description_row is None:
            mf.description_row = row
        else:
            mf.mappings.append(get_mapping(i, row))
    return mf

In [85]:
mfs = []

for path in mf_paths:
    mf = get_mapping_file(path)
    mfs.append(mf)

mfs

[MappingFile{'xls_path': '/Users/jjpr/dev/scratch/mkgu_scratch/data/090609B-4F.xls', 'workbook': <xlrd.book.Book object at 0x104ec3b70>, 'mapping_worksheet': <xlrd.sheet.Sheet object at 0x104fc7a20>, 'comment_lines': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}, 'description_row': [text:'Cerebus mapping for array 090609B-4F/SN0559', empty:'', empty:'', empty:'', empty:'', empty:'', text:'elec', empty:''], 'mappings': [{'column': 8.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '14', 'electrode_label': 'elec99', 'index_in_mapping_sheet': 13}, {'column': 7.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '10', 'electrode_label': 'elec98', 'index_in_mapping_sheet': 14}, {'column': 6.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '8', 'electrode_label': 'elec97', 'index_in_mapping_sheet': 15}, {'column': 5.0, 'row': 9.0, 'bank_name': 'A', 'electrode_number_in_bank': '6', 'electrode_label': 'elec96', 'index_in_mapping_sheet': 16}, {'column': 4.0, 'row': 9.0

In [87]:
[x.description_row[0].value for x in mfs]

['Cerebus mapping for array 090609B-4F/SN0559',
 'Cerebus mapping for array 090609B-5E/SN00547',
 'Cerebus mapping for array 090609B-5F/SN0548',
 'Cerebus mapping for array 090609B-5H/SN0549',
 'Cerebus mapping for array 090609B-8E/SN0552',
 'Cerebus mapping for array 2132-5/SN0656',
 'Cerebus mapping for array 2132-8/SN0657',
 'Cerebus mapping for array 2466-1/SN0658',
 'Cerebus mapping for array 2467-6 / SN 0649',
 'Cerebus mapping for array 5602-2/SN1025-000936']

In [75]:
next((i for i in range (mf.mapping_worksheet.nrows) if not is_comment_row(mf.mapping_worksheet.row(i))), -1)

12

In [74]:
mf.mapping_worksheet.row(12)[0].value

'Cerebus mapping for array 2132-8/SN0657'

In [46]:
mf.mapping_worksheet.nrows

121

In [51]:
mf.__class__.__name__

'MappingFile'

In [47]:
dir(mf)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'mapping_worksheet',
 'workbook',
 'xls_path']

In [52]:
vars(mf)

{'mapping_worksheet': <xlrd.sheet.Sheet at 0x1046b5c18>,
 'workbook': <xlrd.book.Book at 0x10475b940>,
 'xls_path': '/Users/jjpr/dev/scratch/mkgu_scratch/data/5602-2 SN1025-000936.xls'}

In [34]:
try: 
    db
except NameError:
    db_file = os.path.join(data_dir, dated_file_name("db", "array_meta"))
    db_url = "sqlite://" + db_file
    db = dataset.connect(db_url)

In [35]:
db

<Database(sqlite:///Users/jjpr/dev/scratch/mkgu_scratch/data/array_meta_2018-01-29T15:44:59.853746.db)>

In [ ]:
for x in sht.get_rows():
    